# Chapter 32, Introduction to Clustering Models

## Steve Elston

## Incomplete Draft   

**This notebook is a work in progress. Please focus on the the example and Exercises** 

## Loading and Preparing the Dataset   

To explore the basic concepts of k-means and hierarchical clustering we will use a [human resources (HR) dataset from Kaggle](https://www.kaggle.com/jacksonchou/hr-analytics). The business goal is to discover common attributes common to employees who leave a large company prematurely in the data. Since we do not know in advance which sets of common attributes are important in an employees decision to leave, this is a perfect application for clustering methods.   

### Load and explore the data   

As a first step in this analysis, execute the code in the cell below to import the required packages. 

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import pairwise_distances 
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from patsy import dmatrix
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Next, execute the code in the cell below to import the dataset and display the data types of the columns.  

In [ ]:
HR_data = pd.read_csv('../data/HR_comma_sep.csv')
HR_data.dtypes

There are several data types here. The first two variables have floating point numeric values. There are several integer numeric values. And, finally the last two columns appear to have categorical values.  

One question to ask about these data is how many unique levels are there in the columns which are not floating point numbers? To investigate the answer, execute the code in the cell below and examine the results.  

In [ ]:
for col in HR_data.columns[2:]:  
    print('\n' + col)
    print(HR_data[col].unique())

There is a variety of information types encoded in these variables.  
1. Some **ordinal** integer variables. An ordinal variable has an ordered set of values. In this case, several ordinal variables are numeric. But, `salary` is coded by categories.    
2. Several of these variables binary, coded as $[0,1$.    
3. One categorical variable, `sales`, which are job categories. 

To better understand some of the numeric variables it is useful to create histograms. Execute the code in the cell below and examine the histograms displayed.     

In [ ]:
for col in ['satisfaction_level','last_evaluation','average_montly_hours']:
    plt.hist(HR_data.loc[:,col], bins=20, alpha=0.5)
    plt.title('Histogram of ' + col)
    plt.show()

None of these variables are remotely close to a Normal distribution. In fact, these variables look closer to a uniform distribution.      


### Preparing the data   

With some understanding of the variables, we must now prepare the dataset for the analysis. For clustering algorithms we must restrict ourselves to encoding that preserves our ability to compute the required distance metrics. Still there are a number possible distance metrics and encoding options we can choose from.  

For the examples in this notebook we will use data encoding suitable for Euclidean and L1 distances. The choice of Euclidean distance, in particular, dictates that we scale numeric features. Additionally, we will minimize the number of binary features.   
As a first step in this process, we must convert the ordinal variable `salary` to a numeric type. The code in the cell below limits the numeric values to the range $[0,1]$ and maintains the order. Execute this code and note the result 

In [ ]:
salary_levels = {'low':0, 'medium':0.5, 'high':1.0}
HR_data.loc[:,'salary'] = [salary_levels[x] for x in HR_data.loc[:,'salary']]
HR_data.loc[:,'salary'].value_counts()

The `sales` variable presents a particular problem. This variable is categorical and there is no sensible ordering. There is no idea solution. In this case we will recode this variable indicating if the position is in sales or something else. This approach avoids creating a larger number of binary dummy variables. Execute the code in the cell below and examine the results. 

In [ ]:
HR_data.loc[:,'sales'] = [1 if x=='sales' else 0 for x in HR_data.loc[:,'sales']]
HR_data.loc[:,'sales'].value_counts()

About 1/3 of the employees are in sales, so this coding is reasonable from this point of view. 

Finally, we must normalize the numeric variables. Since the distribution is far from Normal, Z-score normalization is likely to be a poor choice. Instead, we will use the [sklearn.preprocessing.MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html). By default, this scale transforms the scale of each variable to be in the $[0.1]$ range. Such scaling in optimal for uniformly distributed variables, and a good choice for variables that are approximately uniform.  

The code in the cell below applies the Min-Max scaler to the numeric columns. Execute this code and examine the results.  

In [ ]:
normalize_cols = ['satisfaction_level','last_evaluation','average_montly_hours','number_project','time_spend_company']
HR_data.loc[:,normalize_cols] = MinMaxScaler().fit_transform(HR_data.loc[:,normalize_cols])
HR_data.head()

The variable values in the data frame are all in the $[0,1]$ range. No variable will dominate the models simply by having numerically large values. 

## K-Means Clustering  

K-means clustering separates a dataset into K clusters of equal variance. The number of clusters, K, is user defined. The basic algorithm has the following steps:
1. A set of K centroids are randomly chosen. 
2. Clusters are formed by minimizing variance within each cluster. This metric is also know as the **within cluster sum of squares** (see further discussion in the section on evaluating clusters). This step partitions the data into clusters with minimum squared distance to the centroid of the cluster. 
3. The centroids are moved to mean of each cluster. The means of each cluster is computed and the centroid is moved to the mean. 
4. Steps 2 and 3 are repeated until a stopping criteria is met. Typically, the algorithm terminates when the within cluster variance decreases only minimally. 
5. The above steps are repeated starting with a random start of step 1. The best set of clusters by within cluster variance and between cluster separation are retained.  

Since K-means clustering relies only on basic linear algebra operations, the method is massively scalable. Out-of-core K-means clustering algorithms are widely used. However, this method assumes equal variance of the clusters, a fairly restrictive assumption. In practice, this criteria is almost never true, and yet K-means clustering still produces useful results. 

### K-means clustering example

Let's try a simple example, applying the k-means algorithm to the prepared HR data. In this case we will use the [sklearn.clustering.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) package to compute cluster assignments using $k=4$.  

The code in the cell below creates a k-means cluster object, and computes the cluster assignments with the `fit_predict` method. A frequency table of the cluster assignments by the value of the `left` variable is then displayed. Execute this code and examine the results.     

In [ ]:
nr.seed(4455)
HR_data['cluster_assignments'] = KMeans(n_clusters=4).fit_predict(HR_data)
## Create a frequency table of the number of employees leaving by cluster assignment  
HR_data.loc[:,['cluster_assignments','left']].value_counts()

Notice that most of the employees who left the company are in a single cluster. Further, this cluster has no employees who did not leave assigned. This appears promising in terms of understanding employees who leave.   

We might consider which factors might be important in compelling employees to leave the company. Visualization of clustering results can be difficult, but with some effort important insight can be gained. Like most data visualization, a number of ideas must be tested. Most visualizations will not be that useful, but a few will be.   

For this case, we will start by displaying a scatter plot matrix of some of the variables. The hue is used to display the cluster assignments. Execute the code and examine the results.  

In [ ]:
def plot_cluster_assignments(df):
    display_columns = ['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'salary', 'cluster_assignments']
    g = sns.PairGrid(HR_data.loc[:,display_columns], hue='cluster_assignments', palette='Set2');
    g.map_diag(sns.kdeplot);
    g.map_offdiag(sns.scatterplot, size=0.1, alpha=0.1);
    g = g.add_legend(fontsize=14, bbox_to_anchor=(1,1));

plot_cluster_assignments(HR_data)

Notice the position of to points for cluster 2, which contains most of the employees who left the company. Many of the points in cluster 2 are in close proximity within several regions of the scatter plots. But, ideally the points of one cluster should in a single compact group.            

## Evaluating cluster models

Now that you have created some clustering models, you are likely wondering how can you evaluate these models and perform model selection. There are a number of metrics you can use to evaluate and compare clustering models. However, you should always keep in mind that the best model, should be selected based on the problem you are trying to solve.

One useful metric for clusters is the **within cluster sum of squares** or **WCSS**. Intuitively, clusters should have minimal dispersion and therefore minimal WCSS. The  

$$WCSS = Min \sum_i \sum_{j\ in\ cluster\ i} ||x_j - c_i||^2 \\
where\\
c_i = center\ of\ ith\ cluster\\ 
and\\
||x_j - c_i|| = distance\ between\ data\ x_j\ and\ center\ c_i
$$

We can use WCSS to compare different cluster models. Models with smaller SSW have tighter clusters and, therefore smaller WCSS. 


> **Note:** WCSS is also referred to as **inertia**, an analogy with classical mechanics in physics. 


The **between cluster sum of squares** or **BCSS** is a related metric. Whereas WCSS measures how tight the clusters are BCSS is a measure of the separation between the clusters. To compute the BCSS, observe that the **total sum of squares** or **TSS** must equal the sum of the WCSS and BCSS:

$$
TSS = BCSS + WCSS\\
where\\
TSS = \sum_i (x_i - \mu)^2\\
where\\
\mu = mean\ of\ all\ data\ samples
$$

Notice that the TSS is just the variance of all data points. The BCSS is then just the difference between TSS and WCSS.


> **Note:** The WCSS and BCSS metrics have the concept of the clustering having multivariate-Normal distributions. Therefore, these metrics are strictly only applicable to K-means cluster. This fact means that WCSS and BCSS are not useful metrics for agglomerative clustering. The SC can be computed using various metrics so is more generally applicable to most clustering methods. 


### Finding the number of clusters

We have investigated our first k-means cluster model with $k = 4$. The next question is, how many clusters ($k$) are optimal. As we are dealing with unsupervised learning models the answer is generally not obvious. 

One approach to determining the number of clusters is to plot the within cluster sum of squares (WCSS) against the number of clusters. The code in the cell below creates just such a plot by the following steps:
1. Any existing cluster assignments are removed from the data frame. It is critical that assignments from other models not bias the training of the new models. 
2. A list of k-means cluster models are computed over a range of $k$ values. 
3. The **inertia** or WCSS is saved for each model computed.  
4. A plot is made of the WCSS vs. the number of clusters. 

Execute this code and examine the result.

In [ ]:
def cluster_search_kmeans(df, nclusts=(2,10)):
    ## If there are cluster assignments in the data frame remove them. 
    if 'cluster_assignments' in df.columns: df.drop(columns='cluster_assignments', inplace=True)
    WCSS=[]
    for n in range(nclusts[0],nclusts[1]+1):   
        temp_model = KMeans(n_clusters=n).fit(df)
        WCSS.append(temp_model.inertia_)
    plt.plot(range(nclusts[0],nclusts[1]+1),WCSS)   
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    
cluster_search_kmeans(HR_data)    

As expected, the slope of the WCSS curve drops rapidly as the number of clusters increases. The question is, where is the best cutoff that leads to an informative, but not over-fit model? This choice is partially subjective. In a more detailed analysis, several models can be compared. In this case, the curve flattens past $k=8$, so we will use an 8 cluster model.  

> **Exercise 27-1:** In the cell below create and execute code to do the following: 
> 1. Compute an 8 cluster model for the HR dataset. Make sure you first remove the `cluster_assignments` column if present.  
> 2. Display a frequency table of cluster assignments by value of the `left` variable

In [ ]:
nr.seed(7722)
## Your code goes below


> Examine the frequency table and answer the following questions:  
> 1. Are there clusters which contain the majority of records for the employees who left the company?  
> 2. Do the foregoing clusters contain any records for employees who stayed with the company?   
> 3. Given your answers to the foregoing questions, do you think the model can provide useful insight into characteristics of customers who leave the company?   

> **Exercise 27-2:** To further evaluate the model you have computed, create execute the code in the cell below to display the pairs plot using the `plot_cluster_assignments` function.   

In [ ]:
## Your code goes here


> Examine the plot. Do the clusters which contain the majority of records of employees who have left the company appear in the reasonably small regions of the scatter plots?

Next, we will display and examine some box plots. Execute the code in the cell below to display box plots of satisfaction level, number of projects, and average monthly hours by cluster assignment. Hue is used to show the left variable. These variables are chosen since clusters containing records of employees who left the company are in a reasonably small range along these axes in the scatter plot matrix. 

In [ ]:
def plot_clusters_by_factor(df, factor='satisfaction_level', type='violin'):
    plt.figure(figsize=(6,4))
    if(type=='violin'):
        sns.violinplot(x='cluster_assignments', y=factor, hue='left', data=df)
    else: 
        sns.boxplot(x='cluster_assignments', y=factor, hue='left', data=df)
    plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
    
plot_clusters_by_factor(HR_data)    
plot_clusters_by_factor(HR_data, factor='number_project')  
plot_clusters_by_factor(HR_data, factor='average_montly_hours')  

We can make some inferences from these results: 
1. Unsurprisingly, employees leaving tend to have the lowest satisfaction scores. However, some of the clusters contain leaving employees with high satisfaction scores.     
2. Employees with the largest number of projects are most likely to leave. The large number of projects and high average monthly hours helps explain why some high satisfaction employees are leaving.  
3. Employees with both the highest and lowest number of monthly hours are likely to leave. As already mentioned, high monthly hours are common in leaving employees. But, some leaving employees have lowest hours and lowest number of projects.    

## Introduction to Hierarchical Clustering

Another widely used form of clustering uses heirarchical modes. These models attempt to divide or partition the data following a heirarchical sequence. 

Heirachical clustering models produce a tree-like organization of the data into a hierarchy of clusters. At the root of the tree, all data cases are in one large cluster. The leaves of the tree each have a cluster with a single data case.

Heirachical clustering models generally use one of two approaches. 

1. **Agglomerative clustering** works from the leaves of the tree toward the root in the following way.
  - All data cases start in their own cluster. 
  - Pairs of clusters are merged to their nearest neighbors over several interations. 
  - The second step is repeated until there is one cluster at the root of the tree. 
2. **Divisive clustering** works from the root of the tree toward the leaves in the following way.
  - All data cases start in a single large cluster. 
  - Clusters are split into two parts in a way that maximizes the distance (or disimilarity) between the clusters. 
  - The second step is repeated until there the leaves of the tree, each with a single data case, is encountered. 
  
  
With the above descriptions of the algorithms in mind, we need to discuss how distance is measured. 

First, we use a **distance metric** to compute the distance between two individual data points. We have already encountered some of the most widely used metrics, **Euclidean** and **Manhattan**. 

Since clusters are typically made up of multiple points we need a way to combine the distances between the points in clusters. We do this through **linkage functions**. There are several commonly used linkage functions. For a distance metric value between two points, $a$ in the one cluster, and $b$ in the other cluster, $D(a,b)$, we can define some common linkage functions:

1. **Ward's method** is a linkage method that uses a minimum variance criteria to select the pairs of smaller clusters to link.  
2. **Maximum or complete-linkage** is the largest value of the distance metric between any pairs of points in the two clusters. 
$$= Max \big( D(a,b) \big)$$
3. **Minimum or single-linkage** is the smallest value of the distance metric between any pairs of points in the two clusters. 
$$= Min \big( D(a,b) \big)$$
4. **Mean or average linkage** is the average of the distance metrics between all pairs of points in the two clusters.
$$= \frac{1}{N_{ab}} \sum D(a,b)\\ 
where\\
N_{ab}\ is\ the\ count\ of\ pair-wise\ distances$$
5. **Centroid linkage** is the distance metric between the centroids between the two clusters. 
$$= |c_1 - c_2|\\
where\\
c_1, c_2\ are\ centroids\ of\ clusters\ 1\ and\ 2$$

As you can imagine, the choice of distance metric and linkage function can significantly change the clustering relationships a model finds. 

### Agglomerative clustering example   

With the foregoing theory in mind, its time to try an example using the HR dataset. We have already explored k-means clustering on this dataset. Recall that the k-means method is only defined for the Euclidean distance metric.  

The agglomerative clustering algorithm, like all hierarchical clustering algorithms, Allows the choice of both a linkage method and a wide range of distance metrics. In this first example we will use complete linkage which uses the maximum distance between any two points within two clusters. We will also use the Manhattan distance metric.  

The code in the cell below creates and fits an agglomerative cluster model and then displays the resulting frequency table. Execute this code and examine the results.

In [ ]:
if 'cluster_assignments' in HR_data.columns: HR_data.drop(columns='cluster_assignments', inplace=True)
model_agglomerative =  AgglomerativeClustering(n_clusters=4, linkage='complete', affinity='manhattan')   
HR_data['cluster_assignments'] = model_agglomerative.fit_predict(HR_data)
HR_data.loc[:,['cluster_assignments','left']].value_counts().sort_index(axis=0, level=0)

There is one cluster containing most of the records of the employees who left the company.   

Execute the code in the cell below to create the scatter plot matrix of some of the variables with the hue showing cluster assignments. 

In [ ]:
plot_cluster_assignments(HR_data)

The records assigned to the cluster with the employees who left the company fall in several groups on the graphs. While it is good most of these records are in one cluster, interpreting the characteristics of these employees in a meaningful way will be difficult at best.      

### Evaluating number of clusters

Given the foregoing results it seems entirely possible that the a different number of clusters might provide better insight into to structure of the data. But how can we evaluate the clusters? With non-Euclidean We cannot use the WCSS. We need an alternative. There are several possibilities     

In this case we will use **Maximum diameter of clusters**. As more clusters are used in a model the clusters become smaller. But, as the model becomes complex, the reduction in the size of the clusters diminishes. The distance must be measured using the clustering distance metric. The maximum diameter of cluster, $C_i$, is:  

$$diameter(C_i) = max_{j,k \in C_i} d(x_j,x_k)$$    



The code in the cell below does the following:    
1. Iterates over each cluster.    
  - The cluster assignment column is deleted.
  - A new model is computed
  - Cluster assignments are made with the predict method. 
  - The `find_max_diameter` function is called. This function iterates over all cluster assignments, computing the maximum diameter of each. The maximum of the maximum is then returned.      
2. Plots the maximum of the diameter of the clusters for each value of $k$.   

Execute the code and examine the results. Since hierarchical clustering is computationally intensive, execution of this code may take some time.  

In [ ]:
def find_max_diameter(df, metric='manhattan'):
    max_diameters = []
    for i in df.loc[:,'cluster_assignments'].unique():
        temp = df[df.loc[:,'cluster_assignments']==i].drop(columns='cluster_assignments')
        max_diameters.append(np.amax(pairwise_distances(temp, metric=metric)))  
    return np.amax(max_diameters)   

def evaluate_agglomerative_clusters(df, metric='manhattan', linkage='complete', nclusts=(2,12)): 
    max_diameters = []
    for n in range(nclusts[0],nclusts[1]+1):
        if 'cluster_assignments' in df.columns: df.drop(columns='cluster_assignments', inplace=True)
        temp_mod=AgglomerativeClustering(n_clusters=n, linkage='complete', affinity=metric)
        df['cluster_assignments'] = temp_mod.fit_predict(df)    
        max_diameters.append(find_max_diameter(df, metric=metric))
    plt.plot(range(nclusts[0],nclusts[1]+1), max_diameters);
    plt.xlabel('Number of clusters')
    plt.ylabel('Maximum cluster diameter')
    return max_diameters     
    
evaluate_agglomerative_clusters(HR_data)    

Examine the plot. The curve is not smooth. A flattening of the curve starting at 9 clusters is evident. Therefore, we will use 9 clusters for the analysis.   

> **Exercise 27-3:** In the cell below create and execute code to do the following:   
> 1. Delete the cluster assignment column if one is present.   
> 2. Compute the 9 cluster model with complete linkage and the Manhattan distance metric. 
> 3. Assign the clusters from the model.   
> 4. Display a frequency table of the `left` variable by cluster assignment.  

In [ ]:
nr.seed(2323)
## Your code goes below. 


> Answer the following questions.   
> 1. Are the majority of the records of employees who left the company in clusters without employees who did not leave the company?   
> 2. Given the above, does this model appear to identify some structure in the data which might provide insight into employees leaving the company.    

To further evaluate this model, execute the code in the cell below and examine the resulting pair-wise scatter plot matrix.    

In [ ]:
plot_cluster_assignments(HR_data)

Examine this plot displaying 9 clusters. The clusters containing the records of employees who left the company are in relatively compact and well-defined regions.   

Next, execute the code in the cell below to further explore the relationships in the clusters.  

In [ ]:
plot_clusters_by_factor(HR_data)    
plot_clusters_by_factor(HR_data, factor='number_project') #, type='box')  
plot_clusters_by_factor(HR_data, factor='average_montly_hours')  

These results look similar to the those from the 8 cluster k-means algorithm. However, in detail there are some differences representing the different algorithms used. Some points to notice here include:   
1. Many employees leaving had low satisfaction.  
2. Employees with large numbers of hours and projects or low numbers of hours and projects are more likely to leave. Notice that the one cluster with both high and low satisfaction employees who left all have high numbers of projects and monthly hours.   

## Using a Different Encoding   

We have applied two types of clustering models using different distance metrics to the HR dataset. Now we will explore the question of what happens when we apply a different encoding to the raw data. Using this scaling we will test using **cosine distance** as the metric. We will proceed cautiously, since generally cosine distance is used for purely categorical data, and this dataset contains several numeric features.     

Specifically, we will expand the `sales`, or job type, into dummy variable columns to encode each job type. The scaling is also modified to ensure the variables are in the range $[-1,1]$. The scaling is often optimal for use with cosine similarity since differences in this range over $\theta = \pi$ rather than just $\theta = \pi/2$.   

The code in the cell below performs the following steps to encode the dataset.  
1. The numeric columns are normalized using min-max scaling.    
2. The ordinal variable `salary_level` is encoded on a $[0,1]$ scale.  
3. The `salary` column is one-hot encoded into binary dummy variables. 
4. The variables are scaled into a range $[-1,1]$ using the transformation $2*value - 1$

Execute the code and examine the resulting data frame.  

In [ ]:
## Get a fresh copy of the data
HR_data_cosine = pd.read_csv('../data/HR_comma_sep.csv')

## Normalize some numeric value columns
normalize_cols = ['satisfaction_level','last_evaluation','average_montly_hours','number_project', 'time_spend_company']
HR_data_cosine.loc[:,normalize_cols] = MinMaxScaler().fit_transform(HR_data_cosine.loc[:,normalize_cols])

## Recode the ordinal salary range to 0-1 
salary_levels = {'low':0.0, 'medium':0.5, 'high':1.0}
HR_data_cosine.loc[:,'salary'] = [salary_levels[x] for x in HR_data_cosine.loc[:,'salary']]

## One-hot encode the categorical columns and normalize by number of categories    
encoded_columns = OneHotEncoder().fit_transform(np.array(HR_data_cosine.loc[:,'sales']).reshape(-1, 1)).toarray()
#encoded_columns = np.divide(encoded_columns, encoded_columns.shape[0]) # float(len(HR_data_cosine.loc[:,'sales'].unique)))
HR_data_cosine[HR_data_cosine.sales.unique()] = encoded_columns

## Finally, transform the feature values to the [-1,1] to improve sensitivity with cosine distance
HR_data_cosine.loc[:,HR_data_cosine.columns] = np.subtract(np.multiply(HR_data_cosine,2.0),1.0)

HR_data_cosine

With the data encoded, next step is to search for the appropriate number of clusters using the cosine metric. Execute the code in the cell below. This may take some time to compute 13 models. 

In [ ]:
evaluate_agglomerative_clusters(HR_data_cosine, metric='cosine', linkage='average', nclusts=(2,14))    

The break in this curve is at 10 clusters.   

> **Exercise 27-4:** Compute a 10 cluster agglomerative model using average linkage, cosine for the distance metric, and display the frequency table of cluster assignment by `left`. 

In [ ]:
nr.seed(8899)
## Your code goes below


> Examine the frequency table and answer the following questions:   
> 1. Are any of the records of the employees who left in clusters separated from records of clusters of employees who have not left. Make sure you account for the encoding of the `left` column.       
> 2. Does the relationships in these clusters lead you to think that this model is over-fit with the clusters fragmented and why?  

#### Copyright 2018, 2019, 2020, 2021, Stephen F Elston. All rights reserved. 